In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
  
from konlpy.tag import *
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 종목 설정
# LG화학, 삼성SDI, SK이노베이션, 고려아연, 포스코케미칼
stock_name = '012450'

In [4]:
news_df = pd.read_csv('./La.csv', usecols=['date_time', 'content', 'token', 'token_okt', 'label'])

news_df['date_time'] = pd.to_datetime(news_df['date_time'].astype(str))
news_df.sort_values('date_time', ignore_index=True, inplace=True)

In [5]:
stock_df = pd.read_csv('./data.csv', usecols=['date_time','change'])
stock_df['date_time'] = pd.to_datetime(stock_df['date_time'])
stock_df.head(2)

,date_time,change
0,2021-07-01 09:01:00,-0.425080
1,2021-07-01 09:02:00,0.320171


In [6]:
start = str(stock_df.iloc[0, 0])
print(start)

2021-07-01 09:01:00


In [7]:
news_df = news_df[news_df[news_df['date_time']==start].index[0]:]
news_df.head(2)

,date_time,content,token,label,token_okt
1,2021-07-01 09:01:00,대한민국 혁신기업 플랫폼미래기술 혁신기업한화솔루션에어로스페이스미래기술 부문에 이름...,"['대한민국', '혁신', '기업', '플랫폼', '미래', '기술', '혁신', ...",0,"['대한민국', '혁신', '기업', '플랫폼', '미래', '기술', '혁신', ..."
2,2021-07-26 10:38:00,서울뉴시스 김제이 기자 한화에어로스페이스는 다음 달 일 오후 시에 투자자 증권사 애...,"['서울', '뉴', '시스', '김', '제이', '기자', '한화', '에', ...",1,"['서울', '뉴시스', '김제', '이', '기자', '한화', '에어로', '스..."


In [ ]:
# news_df['date_time'] = news_df['date_time'] + timedelta(days=1)
# news_df.head(2)

In [9]:
df = news_df.merge(stock_df)
df

,date_time,content,token,label,token_okt,change
0,2021-07-01 09:01:00,대한민국 혁신기업 플랫폼미래기술 혁신기업한화솔루션에어로스페이스미래기술 부문에 이름...,"['대한민국', '혁신', '기업', '플랫폼', '미래', '기술', '혁신', ...",0,"['대한민국', '혁신', '기업', '플랫폼', '미래', '기술', '혁신', ...",-0.425080
1,2021-07-26 10:38:00,서울뉴시스 김제이 기자 한화에어로스페이스는 다음 달 일 오후 시에 투자자 증권사 애...,"['서울', '뉴', '시스', '김', '제이', '기자', '한화', '에', ...",1,"['서울', '뉴시스', '김제', '이', '기자', '한화', '에어로', '스...",0.000000
2,2021-07-30 09:01:00,미래 모빌리티 항공우주 그린수소 에너지 디지털 금융 솔루션 등 신규 사업에도 세계를...,"['미래', '모', '빌리티', '항공', '우주', '그린', '수소', '에너...",1,"['미래', '모빌리티', '항공우주', '그린', '수소', '에너지', '디지털...",0.000000
3,2021-08-06 09:01:00,김철선 기자 한화에어로스페이스는 올해 분기 연결기준 매출 조천억원 영업이익 천억원...,"['김', '철선', '기자', '한화', '에', '어로', '스페이스', '는'...",1,"['김철', '선', '기자', '한화', '에어로', '스페이스', '는', '올...",0.186567
4,2021-08-12 09:01:00,대신증권이 일 한화에어로스페이스의 분기 전 부문 실적 호조가 이어졌다며 목표주가를...,"['대신', '증권', '이', '일', '한화', '에', '어로', '스페이스'...",1,"['대', '신', '증권', '이', '일', '한화', '에어로', '스페이스'...",0.396040
...,...,...,...,...,...,...
1401,2023-06-28 13:31:00,방위산업 부품장비대전 및 첨단국방산업전 대전서 개막무인헬기 등 혁신기술 탑재한 제품...,"['방위', '산업', '부품', '장비', '대전', '및', '첨단', '국방'...",1,"['방위산업', '부품', '장비', '대전', '및', '첨단', '국방', '산...",0.000000
1402,2023-06-28 14:01:00,방산 일자리 박람회 수출상담회 기술교류회 시민 체험행사 등 방위산업 부품장비대전 및...,"['방산', '일자리', '박람회', '수출', '상담', '회', '기술', '교...",1,"['방산', '일자리', '박람회', '수출', '상담', '회', '기술', '교...",0.000000
1403,2023-06-28 14:20:00,한화에어로 임원 인도 현지 언론과 인터뷰잠수함용 리튬이온배터리 수출 적극 검토한화에...,"['한화', '에', '어로', '임원', '인도', '현지', '언론', '과',...",1,"['한화', '에어로', '임원', '인도', '현지', '언론', '과', '인터...",0.000000
1404,2023-06-28 14:20:00,한화에어로 임원 인도 현지 언론과 인터뷰잠수함용 리튬이온배터리 수출 적극 검토한화에...,"['한화', '에', '어로', '임원', '인도', '현지', '언론', '과',...",1,"['한화', '에어로', '임원', '인도', '현지', '언론', '과', '인터...",0.000000


In [10]:
# df.columns = [df.columns[0], df.columns[1], '주가의 날짜', '등락률']

df['content'] = df['content'].astype(str)
df.drop_duplicates('content', inplace=True, ignore_index=True)

print(len(df))

675


In [ ]:
# stopstocks_df = pd.read_csv("./data/종목명.csv", encoding = 'cp949', usecols = [3])
# stopstocks = stopstocks_df['한글 종목약명'].tolist()

In [ ]:
# for stopstock in tqdm(stopstocks) :
#     str_expr = f"Title.str.contains('{stopstock}')"
#     i = df.query(str_expr).index
#     df.loc[i, 'Title'] = df.loc[i, 'Title'].str.replace(stopstock, '')

In [13]:
# df['updown'] = 1
# df.loc[df.query('등락률 < 0').index, 'updown'] = -1
df['label'].value_counts()

label
1    392
0    283
Name: count, dtype: int64

In [14]:
df.isnull().sum()

date_time    0
content      0
token        0
label        0
token_okt    0
change       0
dtype: int64

In [15]:
n_ =[]
okt =Okt()

In [16]:
# 기사 제목 앞 뒤로 괄호/대괄호 지우기
df['content'] = df['content'].str.replace(r'[\(\[].*?[\)\]]', '', regex = True)

In [17]:
df

,date_time,content,token,label,token_okt,change
0,2021-07-01 09:01:00,대한민국 혁신기업 플랫폼미래기술 혁신기업한화솔루션에어로스페이스미래기술 부문에 이름...,"['대한민국', '혁신', '기업', '플랫폼', '미래', '기술', '혁신', ...",0,"['대한민국', '혁신', '기업', '플랫폼', '미래', '기술', '혁신', ...",-0.425080
1,2021-07-26 10:38:00,서울뉴시스 김제이 기자 한화에어로스페이스는 다음 달 일 오후 시에 투자자 증권사 애...,"['서울', '뉴', '시스', '김', '제이', '기자', '한화', '에', ...",1,"['서울', '뉴시스', '김제', '이', '기자', '한화', '에어로', '스...",0.000000
2,2021-07-30 09:01:00,미래 모빌리티 항공우주 그린수소 에너지 디지털 금융 솔루션 등 신규 사업에도 세계를...,"['미래', '모', '빌리티', '항공', '우주', '그린', '수소', '에너...",1,"['미래', '모빌리티', '항공우주', '그린', '수소', '에너지', '디지털...",0.000000
3,2021-08-06 09:01:00,김철선 기자 한화에어로스페이스는 올해 분기 연결기준 매출 조천억원 영업이익 천억원...,"['김', '철선', '기자', '한화', '에', '어로', '스페이스', '는'...",1,"['김철', '선', '기자', '한화', '에어로', '스페이스', '는', '올...",0.186567
4,2021-08-12 09:01:00,대신증권이 일 한화에어로스페이스의 분기 전 부문 실적 호조가 이어졌다며 목표주가를...,"['대신', '증권', '이', '일', '한화', '에', '어로', '스페이스'...",1,"['대', '신', '증권', '이', '일', '한화', '에어로', '스페이스'...",0.396040
...,...,...,...,...,...,...
670,2023-06-28 11:07:00,방산 일자리 박람회 수출상담회 기술교류회 시민 체험행사까지 다채로운 프로그램 마련...,"['방산', '일자리', '박람회', '수출', '상담', '회', '기술', '교...",1,"['방산', '일자리', '박람회', '수출', '상담', '회', '기술', '교...",0.000000
671,2023-06-28 11:22:00,현대중공업이 국제해양방위산업전 에서 선보인 차세대 한국형 구축함 모형한화오션이 국제...,"['현대', '중공업', '이', '국제', '해양', '방위', '산업', '전'...",0,"['현', '대', '중공업', '이', '국제', '해', '양', '방위산업',...",-0.076982
672,2023-06-28 13:31:00,방위산업 부품장비대전 및 첨단국방산업전 대전서 개막무인헬기 등 혁신기술 탑재한 제품...,"['방위', '산업', '부품', '장비', '대전', '및', '첨단', '국방'...",1,"['방위산업', '부품', '장비', '대전', '및', '첨단', '국방', '산...",0.000000
673,2023-06-28 14:01:00,방산 일자리 박람회 수출상담회 기술교류회 시민 체험행사 등 방위산업 부품장비대전 및...,"['방산', '일자리', '박람회', '수출', '상담', '회', '기술', '교...",1,"['방산', '일자리', '박람회', '수출', '상담', '회', '기술', '교...",0.000000


In [20]:
from tqdm import tqdm, tqdm_pandas

for i in tqdm(range(len(df))) : 
    # 바뀌어진 기사 제목에서 okt로 명사만 뽑기
    n_.append(' '.join(okt.nouns(df.loc[i,'content'])))

df['nouns'] = n_
df = df[df['nouns'] != '']
df.head(3)

100%|██████████| 675/675 [00:24<00:00, 27.60it/s]


,date_time,content,token,label,token_okt,change,nouns
0,2021-07-01 09:01:00,대한민국 혁신기업 플랫폼미래기술 혁신기업한화솔루션에어로스페이스미래기술 부문에 이름...,"['대한민국', '혁신', '기업', '플랫폼', '미래', '기술', '혁신', ...",0,"['대한민국', '혁신', '기업', '플랫폼', '미래', '기술', '혁신', ...",-0.42508,대한민국 혁신 기업 플랫폼 미래 기술 혁신 기업 한화 솔루션 에어로 스페이스 미래 ...
1,2021-07-26 10:38:00,서울뉴시스 김제이 기자 한화에어로스페이스는 다음 달 일 오후 시에 투자자 증권사 애...,"['서울', '뉴', '시스', '김', '제이', '기자', '한화', '에', ...",1,"['서울', '뉴시스', '김제', '이', '기자', '한화', '에어로', '스...",0.00000,서울 뉴시스 김제 기자 한화 에어로 스페이스 다음 달 일 오후 시 투자자 증권사 애...
2,2021-07-30 09:01:00,미래 모빌리티 항공우주 그린수소 에너지 디지털 금융 솔루션 등 신규 사업에도 세계를...,"['미래', '모', '빌리티', '항공', '우주', '그린', '수소', '에너...",1,"['미래', '모빌리티', '항공우주', '그린', '수소', '에너지', '디지털...",0.00000,미래 모빌리티 항공우주 그린 수소 에너지 디지털 금융 솔루션 등 신규 사업 세계 상...


In [21]:
### 여기부터 사전생성 및 모델링일듯
vocab = {}

for nouns in tqdm(df['nouns']) : 
    nouns = nouns.split(' ')
    
    for n in nouns :
        if n in vocab or len(n) <= 1 :
            pass
        else :
            vocab[n] = 0
    
print(len(vocab))
print(vocab)

100%|██████████| 675/675 [00:00<00:00, 32436.50it/s]

6458
{'대한민국': 0, '혁신': 0, '기업': 0, '플랫폼': 0, '미래': 0, '기술': 0, '한화': 0, '솔루션': 0, '에어로': 0, '스페이스': 0, '부문': 0, '이름': 0, '하이브': 0, '유일': 0, '리노': 0, '공업': 0, '더존': 0, '비즈': 0, '온솔': 0, '브레인': 0, '중견': 0, '선정': 0, '엔씨소프트': 0, '넷마블': 0, '게임': 0, '업체': 0, '최고경영자': 0, '로부터': 0, '평가': 0, '엔터테인먼트': 0, '포함': 0, '그룹': 0, '계열': 0, '다수': 0, '포진': 0, '반도체': 0, '사용': 0, '소켓': 0, '생산': 0, '정보기술': 0, '분야': 0, '글로벌': 0, '시장': 0, '점유': 0, '시가총액': 0, '조억원': 0, '운영': 0, '온도': 0, '국내': 0, '최대': 0, '전사적자원관리': 0, '시스템': 0, '클라우드': 0, '판매': 0, '거래': 0, '중소기업': 0, '대기업': 0, '활용': 0, '대표': 0, '화장품': 0, '여러': 0, '영역': 0, '사업': 0, '확장': 0, '지난': 0, '유니버스': 0, '출시': 0, '개국': 0, '서비스': 0, '시작': 0, '브레이브걸스': 0, '몬스타': 0, '엑스': 0, '다니엘': 0, '마이걸': 0, '인기': 0, '아이돌': 0, '뮤직비디오': 0, '화보': 0, '독점': 0, '공급': 0, '최근': 0, '신설': 0, '법인': 0, '힐러': 0, '설립': 0, '건강': 0, '기능': 0, '식품': 0, '음료': 0, '생활용품': 0, '제조': 0, '판매업': 0, '진행': 0, '인수': 0, '코웨이': 0, '시너지': 0, '효과': 0, '전략': 0, '개인': 0, '맞춤': 0, '미용': 0, '제공': 0, '계획': 0,

In [22]:
up = len(df[df['label'] == 1])
down = len(df[df['label'] == 0])
up_ratio = up / (up + down)
down_ratio = down / (up + down)

print('up_ratio : ', up_ratio)
print('down_ratio : ', down_ratio)

up_ratio :  0.5807407407407408
down_ratio :  0.4192592592592593


In [23]:
for i, word in tqdm(enumerate(df['nouns'])) :
    word = word.split(' ')
    
    for w in word :
        if df.iloc[i]['label'] == 1 :
            if len(w) >= 2 :
                vocab[w] +=  down_ratio
        else :
            if len(w) >= 2 :
                vocab[w] -=  up_ratio

675it [00:04, 165.02it/s]


In [24]:
vocab

{'대한민국': 32.885925925925974,
 '혁신': 1.3170370370370434,
 '기업': 12.634074074074078,
 '플랫폼': -0.09925925925925866,
 '미래': 6.622222222222254,
 '기술': 42.06370370370345,
 '한화': 354.36296296297917,
 '솔루션': 4.4948148148148475,
 '에어로': 35.18666666666342,
 '스페이스': 49.180740740738074,
 '부문': -6.197037037036955,
 '이름': 4.124444444444445,
 '하이브': -4.065185185185186,
 '유일': 4.189629629629629,
 '리노': -1.7422222222222223,
 '공업': -5.807407407407407,
 '더존': -2.322962962962963,
 '비즈': -4.645925925925926,
 '온솔': -0.5807407407407408,
 '브레인': -1.1614814814814816,
 '중견': -3.54962962962963,
 '선정': 7.625185185185208,
 '엔씨소프트': -3.4844444444444447,
 '넷마블': -2.903703703703704,
 '게임': -2.4844444444444447,
 '업체': 20.911111111111143,
 '최고경영자': -1.1614814814814816,
 '로부터': -3.2607407407407414,
 '평가': 4.013333333333362,
 '엔터테인먼트': -2.322962962962963,
 '포함': 2.245925925925931,
 '그룹': 51.826666666666476,
 '계열': -24.788148148148096,
 '다수': -1.1333333333333337,
 '포진': -0.5807407407407408,
 '반도체': -3.2948148148148126,
 '

In [25]:
robustScaler = RobustScaler()

vocab_v = [v for v in vocab.values()]

scaled_list = robustScaler.fit_transform(np.array(vocab_v).reshape(-1, 1))

scaled_list = scaled_list.reshape(1,-1)

for idx, v in tqdm(enumerate(vocab_v)) :
    vocab_v[idx] = scaled_list[0][idx]

for i, k in tqdm(enumerate(vocab.keys())) :
    vocab[k] = vocab_v[i]

6458it [00:00, 1996227.82it/s]
6458it [00:00, 2636956.31it/s]


In [26]:
vocab

{'대한민국': 22.875782881002124,
 '혁신': 0.6325678496868522,
 '기업': 8.606471816283928,
 '플랫폼': -0.36534446764091816,
 '미래': 4.3705636743215255,
 '기술': 29.342379958246173,
 '한화': 249.38622129437468,
 '솔루션': 2.871607515657643,
 '에어로': 24.496868475989366,
 '스페이스': 34.35699373695011,
 '부문': -4.661795407098063,
 '이름': 2.610647181628393,
 '하이브': -3.159707724425888,
 '유일': 2.6565762004175366,
 '리노': -1.522964509394572,
 '공업': -4.387265135699374,
 '더존': -1.932150313152401,
 '비즈': -3.568893528183717,
 '온솔': -0.7045929018789144,
 '브레인': -1.1137787056367432,
 '중견': -2.796450939457203,
 '선정': 5.077244258872667,
 '엔씨소프트': -2.7505219206680587,
 '넷마블': -2.34133611691023,
 '게임': -2.045929018789144,
 '업체': 14.438413361169125,
 '최고경영자': -1.1137787056367432,
 '로부터': -2.5929018789144056,
 '평가': 2.5323590814196444,
 '엔터테인먼트': -1.932150313152401,
 '포함': 1.287056367432154,
 '그룹': 36.22129436325665,
 '계열': -17.760960334029193,
 '다수': -1.0939457202505223,
 '포진': -0.7045929018789144,
 '반도체': -2.6169102296450926,
 '사

In [27]:
total = []

for i, word in tqdm(enumerate(df['nouns'])) :
    word = word.split(' ')
    sent_score = 0
    
    for w in word :
        if len(w) >= 2 and w in vocab :
            sent_score += vocab[w]
    
    total.append(sent_score / len(word))

df['sent_score'] = total


675it [00:00, 18881.04it/s]


,date_time,content,token,label,token_okt,change,nouns,sent_score
0,2021-07-01 09:01:00,대한민국 혁신기업 플랫폼미래기술 혁신기업한화솔루션에어로스페이스미래기술 부문에 이름...,"['대한민국', '혁신', '기업', '플랫폼', '미래', '기술', '혁신', ...",0,"['대한민국', '혁신', '기업', '플랫폼', '미래', '기술', '혁신', ...",-0.42508,대한민국 혁신 기업 플랫폼 미래 기술 혁신 기업 한화 솔루션 에어로 스페이스 미래 ...,9.129702
1,2021-07-26 10:38:00,서울뉴시스 김제이 기자 한화에어로스페이스는 다음 달 일 오후 시에 투자자 증권사 애...,"['서울', '뉴', '시스', '김', '제이', '기자', '한화', '에', ...",1,"['서울', '뉴시스', '김제', '이', '기자', '한화', '에어로', '스...",0.00000,서울 뉴시스 김제 기자 한화 에어로 스페이스 다음 달 일 오후 시 투자자 증권사 애...,8.002531
2,2021-07-30 09:01:00,미래 모빌리티 항공우주 그린수소 에너지 디지털 금융 솔루션 등 신규 사업에도 세계를...,"['미래', '모', '빌리티', '항공', '우주', '그린', '수소', '에너...",1,"['미래', '모빌리티', '항공우주', '그린', '수소', '에너지', '디지털...",0.00000,미래 모빌리티 항공우주 그린 수소 에너지 디지털 금융 솔루션 등 신규 사업 세계 상...,17.447373


In [28]:
df.head(30)

,date_time,content,token,label,token_okt,change,nouns,sent_score
0,2021-07-01 09:01:00,대한민국 혁신기업 플랫폼미래기술 혁신기업한화솔루션에어로스페이스미래기술 부문에 이름...,"['대한민국', '혁신', '기업', '플랫폼', '미래', '기술', '혁신', ...",0,"['대한민국', '혁신', '기업', '플랫폼', '미래', '기술', '혁신', ...",-0.425080,대한민국 혁신 기업 플랫폼 미래 기술 혁신 기업 한화 솔루션 에어로 스페이스 미래 ...,9.129702
1,2021-07-26 10:38:00,서울뉴시스 김제이 기자 한화에어로스페이스는 다음 달 일 오후 시에 투자자 증권사 애...,"['서울', '뉴', '시스', '김', '제이', '기자', '한화', '에', ...",1,"['서울', '뉴시스', '김제', '이', '기자', '한화', '에어로', '스...",0.000000,서울 뉴시스 김제 기자 한화 에어로 스페이스 다음 달 일 오후 시 투자자 증권사 애...,8.002531
2,2021-07-30 09:01:00,미래 모빌리티 항공우주 그린수소 에너지 디지털 금융 솔루션 등 신규 사업에도 세계를...,"['미래', '모', '빌리티', '항공', '우주', '그린', '수소', '에너...",1,"['미래', '모빌리티', '항공우주', '그린', '수소', '에너지', '디지털...",0.000000,미래 모빌리티 항공우주 그린 수소 에너지 디지털 금융 솔루션 등 신규 사업 세계 상...,17.447373
3,2021-08-06 09:01:00,김철선 기자 한화에어로스페이스는 올해 분기 연결기준 매출 조천억원 영업이익 천억원...,"['김', '철선', '기자', '한화', '에', '어로', '스페이스', '는'...",1,"['김철', '선', '기자', '한화', '에어로', '스페이스', '는', '올...",0.186567,김철 선 기자 한화 에어로 스페이스 올해 분기 연결 기준 매출 조천억원 영업 이익 ...,14.703515
4,2021-08-12 09:01:00,대신증권이 일 한화에어로스페이스의 분기 전 부문 실적 호조가 이어졌다며 목표주가를...,"['대신', '증권', '이', '일', '한화', '에', '어로', '스페이스'...",1,"['대', '신', '증권', '이', '일', '한화', '에어로', '스페이스'...",0.396040,증권 일 한화 에어로 스페이스 분기 전 부문 실적 호조 목표 주가 기존 만원 만원 ...,3.309893
5,2021-08-24 09:01:00,임금 청구 소송 상고심에서 원고 승소 판결 확정 금속노조 회사는 조합원에 사과해야 ...,"['임금', '청구', '소송', '상고심', '에서', '원고', '승소', '판...",1,"['임금', '청구', '소송', '상', '고심', '에서', '원고', '승소'...",0.522466,임금 청구 소송 상 고심 원고 승소 판결 확정 금속노조 회사 조합원 사과 소재 한화...,8.886407
6,2021-09-07 13:30:00,계획대로라면 오는 월일 발사 예정된 누리호한화에어로스페이스에서 만든 엔진 기 들어가...,"['계획', '대로', '라면', '오', '는', '월일', '발사', '예정',...",0,"['계획', '대로', '라면', '오는', '월일', '발사', '예정', '된'...",-0.197239,계획 라면 월일 발사 예정 누리 호한 에어로 스페이스 엔진 기 당장 우주 사업 수익...,9.767109
7,2021-09-09 14:27:00,지난 일 한화에어로스페이스와 개 정부 출연 연구기관의 우주 현지자원활용 업무협약 체...,"['지나', 'ㄴ', '일', '한화', '에', '어로', '스페이스', '와',...",1,"['지난', '일', '한화', '에어로', '스페이스', '와', '개', '정부...",0.100200,지난 일 한화 에어로 스페이스 개 정부 출연 연구기관 우주 현지 자원 활용 업무 협...,9.517556
8,2021-09-10 09:17:00,한화에어로스페이스는 지난 일 경기도 성남시 판교삼평공원에서 성남시 자원봉사센터와 ...,"['한화', '에', '어로', '스페이스', '는', '지나', 'ㄴ', '일',...",1,"['한화', '에어로', '스페이스', '는', '지난', '일', '경기도', '...",0.200000,한화 에어로 스페이스 지난 일 경기도 성남시 판교 평 공원 성남시 자원봉사 센터 공...,20.201835
9,2021-09-10 09:55:00,문창석 기자 한화에어로스페이스는 지난 일 경기도 성남시 판교 삼평공원에서 성남시 ...,"['문', '창', '석', '기자', '한화', '에', '어로', '스페이스',...",0,"['문', '창석', '기자', '한화', '에어로', '스페이스', '는', '지...",-0.201207,창석 기자 한화 에어로 스페이스 지난 일 경기도 성남시 판교 평 공원 성남시 자원봉...,17.603157


In [29]:
desc = df['sent_score'].describe()
desc

count    675.000000
mean      14.151946
std       11.317242
min       -7.493195
25%        6.258542
50%       11.943133
75%       19.755114
max       55.673417
Name: sent_score, dtype: float64

In [30]:
median = desc['50%']
df['sent_label'] = 0

df.loc[df.query(f'sent_score >= {median}').index, 'sent_label'] = 1 


,date_time,content,token,label,token_okt,change,nouns,sent_score,sent_label
0,2021-07-01 09:01:00,대한민국 혁신기업 플랫폼미래기술 혁신기업한화솔루션에어로스페이스미래기술 부문에 이름...,"['대한민국', '혁신', '기업', '플랫폼', '미래', '기술', '혁신', ...",0,"['대한민국', '혁신', '기업', '플랫폼', '미래', '기술', '혁신', ...",-0.42508,대한민국 혁신 기업 플랫폼 미래 기술 혁신 기업 한화 솔루션 에어로 스페이스 미래 ...,9.129702,0
1,2021-07-26 10:38:00,서울뉴시스 김제이 기자 한화에어로스페이스는 다음 달 일 오후 시에 투자자 증권사 애...,"['서울', '뉴', '시스', '김', '제이', '기자', '한화', '에', ...",1,"['서울', '뉴시스', '김제', '이', '기자', '한화', '에어로', '스...",0.00000,서울 뉴시스 김제 기자 한화 에어로 스페이스 다음 달 일 오후 시 투자자 증권사 애...,8.002531,0
2,2021-07-30 09:01:00,미래 모빌리티 항공우주 그린수소 에너지 디지털 금융 솔루션 등 신규 사업에도 세계를...,"['미래', '모', '빌리티', '항공', '우주', '그린', '수소', '에너...",1,"['미래', '모빌리티', '항공우주', '그린', '수소', '에너지', '디지털...",0.00000,미래 모빌리티 항공우주 그린 수소 에너지 디지털 금융 솔루션 등 신규 사업 세계 상...,17.447373,1


In [31]:
df.head(30)

,date_time,content,token,label,token_okt,change,nouns,sent_score,sent_label
0,2021-07-01 09:01:00,대한민국 혁신기업 플랫폼미래기술 혁신기업한화솔루션에어로스페이스미래기술 부문에 이름...,"['대한민국', '혁신', '기업', '플랫폼', '미래', '기술', '혁신', ...",0,"['대한민국', '혁신', '기업', '플랫폼', '미래', '기술', '혁신', ...",-0.425080,대한민국 혁신 기업 플랫폼 미래 기술 혁신 기업 한화 솔루션 에어로 스페이스 미래 ...,9.129702,0
1,2021-07-26 10:38:00,서울뉴시스 김제이 기자 한화에어로스페이스는 다음 달 일 오후 시에 투자자 증권사 애...,"['서울', '뉴', '시스', '김', '제이', '기자', '한화', '에', ...",1,"['서울', '뉴시스', '김제', '이', '기자', '한화', '에어로', '스...",0.000000,서울 뉴시스 김제 기자 한화 에어로 스페이스 다음 달 일 오후 시 투자자 증권사 애...,8.002531,0
2,2021-07-30 09:01:00,미래 모빌리티 항공우주 그린수소 에너지 디지털 금융 솔루션 등 신규 사업에도 세계를...,"['미래', '모', '빌리티', '항공', '우주', '그린', '수소', '에너...",1,"['미래', '모빌리티', '항공우주', '그린', '수소', '에너지', '디지털...",0.000000,미래 모빌리티 항공우주 그린 수소 에너지 디지털 금융 솔루션 등 신규 사업 세계 상...,17.447373,1
3,2021-08-06 09:01:00,김철선 기자 한화에어로스페이스는 올해 분기 연결기준 매출 조천억원 영업이익 천억원...,"['김', '철선', '기자', '한화', '에', '어로', '스페이스', '는'...",1,"['김철', '선', '기자', '한화', '에어로', '스페이스', '는', '올...",0.186567,김철 선 기자 한화 에어로 스페이스 올해 분기 연결 기준 매출 조천억원 영업 이익 ...,14.703515,1
4,2021-08-12 09:01:00,대신증권이 일 한화에어로스페이스의 분기 전 부문 실적 호조가 이어졌다며 목표주가를...,"['대신', '증권', '이', '일', '한화', '에', '어로', '스페이스'...",1,"['대', '신', '증권', '이', '일', '한화', '에어로', '스페이스'...",0.396040,증권 일 한화 에어로 스페이스 분기 전 부문 실적 호조 목표 주가 기존 만원 만원 ...,3.309893,0
5,2021-08-24 09:01:00,임금 청구 소송 상고심에서 원고 승소 판결 확정 금속노조 회사는 조합원에 사과해야 ...,"['임금', '청구', '소송', '상고심', '에서', '원고', '승소', '판...",1,"['임금', '청구', '소송', '상', '고심', '에서', '원고', '승소'...",0.522466,임금 청구 소송 상 고심 원고 승소 판결 확정 금속노조 회사 조합원 사과 소재 한화...,8.886407,0
6,2021-09-07 13:30:00,계획대로라면 오는 월일 발사 예정된 누리호한화에어로스페이스에서 만든 엔진 기 들어가...,"['계획', '대로', '라면', '오', '는', '월일', '발사', '예정',...",0,"['계획', '대로', '라면', '오는', '월일', '발사', '예정', '된'...",-0.197239,계획 라면 월일 발사 예정 누리 호한 에어로 스페이스 엔진 기 당장 우주 사업 수익...,9.767109,0
7,2021-09-09 14:27:00,지난 일 한화에어로스페이스와 개 정부 출연 연구기관의 우주 현지자원활용 업무협약 체...,"['지나', 'ㄴ', '일', '한화', '에', '어로', '스페이스', '와',...",1,"['지난', '일', '한화', '에어로', '스페이스', '와', '개', '정부...",0.100200,지난 일 한화 에어로 스페이스 개 정부 출연 연구기관 우주 현지 자원 활용 업무 협...,9.517556,0
8,2021-09-10 09:17:00,한화에어로스페이스는 지난 일 경기도 성남시 판교삼평공원에서 성남시 자원봉사센터와 ...,"['한화', '에', '어로', '스페이스', '는', '지나', 'ㄴ', '일',...",1,"['한화', '에어로', '스페이스', '는', '지난', '일', '경기도', '...",0.200000,한화 에어로 스페이스 지난 일 경기도 성남시 판교 평 공원 성남시 자원봉사 센터 공...,20.201835,1
9,2021-09-10 09:55:00,문창석 기자 한화에어로스페이스는 지난 일 경기도 성남시 판교 삼평공원에서 성남시 ...,"['문', '창', '석', '기자', '한화', '에', '어로', '스페이스',...",0,"['문', '창석', '기자', '한화', '에어로', '스페이스', '는', '지...",-0.201207,창석 기자 한화 에어로 스페이스 지난 일 경기도 성남시 판교 평 공원 성남시 자원봉...,17.603157,1


In [33]:
df.dropna(how='any',inplace= True)

df['tokenized'] = df['content'].apply(okt.morphs)

X_train = df['tokenized'].values
Y_train = df['sent_label'].values

X_train

array([list(['대한민국', '혁신', '기업', '플랫폼', '미래', '기술', '혁신', '기업', '한화', '솔루션', '에어로', '스페이스', '미래', '기술', '부문', '에', '이름', '올려', '하이브', '엔', '터', '기업', '으로는', '유일', '리노', '공업', '더존', '비즈', '온솔', '브레인', '등', '기술', '력', '을', '앞세운', '중견', '기업', '들', '이', '대한민국', '혁신', '기업', '곳', '에', '선정', '됐다', '엔씨소프트', '넷마블', '등', '게임', '업체', '도', '혁신', '기업', '최고경영자', '들', '로부터', '좋은', '평가', '를', '받았고', '하이브', '는', '엔터테인먼트', '기업', '중', '유일하게', '포함', '됐다', '한화', '에어로', '스페이스', '한화', '솔루션', '등', '한화', '그룹', '계열', '사들이', '혁신', '미래', '기술', '기업', '에', '다수', '포진', '한', '것', '도', '눈', '에', '띄었다', '반도체', '검', '사용', '핀', '과', '소켓', '등', '을', '생산', '하는', '리노', '공업', '은', '정보기술', '분야', '의', '혁신', '기업', '으로', '뽑혔다', '리노', '공업', '의', '핀', '글로벌', '시장', '점유', '율', '은', '가', '넘는다', '중견', '기업', '으로', '드물게', '시가총액', '이', '조억원', '이', '넘는다', '기업', '운영', '솔루션', '전', '문', '업체', '인', '더존', '비즈', '온도', '부문', '혁신', '기업', '에', '이름', '을', '올렸다', '더존', '비즈', '온', '은', '국내', '최대', '전사적자원관리', '시스템', '과', '클라우드', '등', '기업', '운영', '솔루션'

In [34]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# 빈도 수가 낮은 단어 제거
threshold = 3

words_cnt = len(tokenizer.word_index)
rare_cnt = 0
words_freq = 0
rare_freq = 0

for key, value in tqdm(tokenizer.word_counts.items()) :
    words_freq = words_freq + value

    if value < threshold :
        rare_cnt += 1
        rare_freq = rare_freq + value


100%|██████████| 10816/10816 [00:00<00:00, 3315713.50it/s]

전체 단어 수 : 10816
빈도가 2 이하인 희귀 단어 수 : 5957
희귀 단어 비율 : 55.07581360946746
희귀 단어 등장 빈도 비율 : 3.1657063995698906


In [36]:
print("전체 단어 수 :", words_cnt)
print("빈도가 {} 이하인 희귀 단어 수 : {}".format(threshold - 1, rare_cnt))
print("희귀 단어 비율 : {}".format((rare_cnt / words_cnt) * 100))
print("희귀 단어 등장 빈도 비율 : {}".format((rare_freq / words_freq) * 100))

전체 단어 수 : 10816
빈도가 2 이하인 희귀 단어 수 : 5957
희귀 단어 비율 : 55.07581360946746
희귀 단어 등장 빈도 비율 : 3.1657063995698906


In [35]:
vocab_size = words_cnt - rare_cnt + 2
print(vocab_size)

4861


In [37]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)

In [38]:
max_len = max(len(i) for i in X_train)
print(max_len)

6476


In [39]:
# 패딩
X_train = pad_sequences(X_train, maxlen = max_len)

In [40]:
model = Sequential()

model.add(Embedding(vocab_size, 100))

model.add(Bidirectional(LSTM(100)))

model.add(Dense(1, activation = 'sigmoid'))

es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 4)
mc = ModelCheckpoint('best_model.h5', monitor = 'val_acc', mode = 'max', verbose = 2, save_best_only = True)

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])

history = model.fit(X_train, Y_train, epochs = 15, callbacks = [es, mc], batch_size = 256, validation_split = 0.2)

model.summary()

Epoch 1/15


In [ ]:
# 저장된 모델 불러오기, 평가
loaded_model = load_model('best_model.h5')
loaded_model.evaluate(X_train, Y_train)

In [ ]:
hist_dict = history.history
loss = hist_dict['loss']
val_loss = hist_dict['val_loss']
acc = hist_dict['acc']
val_acc = hist_dict['val_acc']

plt.plot(loss, 'b--', label = 'training loss')
plt.plot(val_loss, 'r--', label = 'validation loss')
plt.legend()
plt.grid()

plt.figure()
plt.plot(acc, 'b--', label = 'training accuracy')
plt.plot(val_acc, 'r--', label = 'validation accuracy')
plt.legend()
plt.grid()

plt.show()

In [ ]:
def sentiment_predict(new_sentence) :
    
    new_token = okt.morphs(new_sentence)
    new_sequences = tokenizer.texts_to_sequences([new_token])
    new_pad = pad_sequences(new_sequences, maxlen = max_len)
    score = float(loaded_model.predict(new_pad))

    if score >= 0.5 :
        print("{} -> 긍정 ({:.2f}%)".format(new_sentence, score * 100))
    else :
        print("{} -> 부정 ({:.2f}%)".format(new_sentence, (1 - score) * 100))

In [ ]:
sentiment_predict("2차전지株, 실적 부진해도 상승하는 까닭")